In [14]:
import cv2
import mediapipe as mp
import numpy as np
import math
import os

In [15]:
mp_hands = mp.solutions.hands

In [16]:
def get_hand_image(image):
    width = image.shape[1]
    height = image.shape[0]
    with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = hands.process(image)
        image.flags.writeable = True
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                landmarks = np.array([[lm.x * width, lm.y * height] for lm in hand_landmarks.landmark])
                hand_bbox = cv2.boundingRect(landmarks.astype(np.float32))
                # extract hand image
                hand_image = image[hand_bbox[1]:hand_bbox[1]+hand_bbox[3], hand_bbox[0]:hand_bbox[0]+hand_bbox[2]]
                return hand_image
        
        

In [17]:
def create_seq_of_frames(vidPath):
    sequence = []
    vid = cv2.VideoCapture(vidPath)
    total_frames = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
    print("There are {} frames to be processed ".format(total_frames))
    
    # Determine the step size to select frames evenly
    step = max(total_frames // 12, 1)
    
    for frame_num in range(0, total_frames, step):
        vid.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        _, frame = vid.read()
        frame = cv2.resize(frame, (64, 64))
        # print("Processing frame {}".format(frame_num))
        if get_hand_image(frame) is not None:
            sequence.append(get_hand_image(frame) / 255)
        # sequence.append(get_hand_image(frame) / 255)
    # Apply pre-padding if necessary
    while len(sequence) < 12:
        sequence.insert(0, np.zeros((64, 64, 3)))
    
    vid.release()
    return sequence[:12] 

In [18]:
def create_dataset(parentFolder,dataset):
    for file in os.listdir(parentFolder):
        filePath = "{}/{}".format(parentFolder,file)
        
        print("Processing " + filePath)
        dataset.append(create_seq_of_frames(filePath))
    return dataset

In [19]:
x = []
y = []

In [20]:
x = create_dataset("../Dataset2/Correct",x)
x = create_dataset("../Dataset2/Incorrect",x)

Processing ../Dataset2/Correct/20240330_184451.mp4
There are 47 frames to be processed 
Processing ../Dataset2/Correct/20240330_184455.mp4
There are 45 frames to be processed 
Processing ../Dataset2/Correct/20240330_184458.mp4
There are 56 frames to be processed 
Processing ../Dataset2/Correct/20240330_184502.mp4
There are 53 frames to be processed 
Processing ../Dataset2/Correct/20240330_184505.mp4
There are 67 frames to be processed 
Processing ../Dataset2/Correct/20240330_184708.mp4
There are 46 frames to be processed 
Processing ../Dataset2/Correct/20240330_184714.mp4
There are 47 frames to be processed 
Processing ../Dataset2/Correct/20240330_184717.mp4
There are 58 frames to be processed 
Processing ../Dataset2/Correct/20240330_184720.mp4
There are 60 frames to be processed 
Processing ../Dataset2/Correct/20240330_184724.mp4
There are 54 frames to be processed 
Processing ../Dataset2/Correct/20240330_184728.mp4
There are 57 frames to be processed 
Processing ../Dataset2/Correct/2

In [21]:
y = [1]*60 + [0]*60
y

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
import pickle

In [23]:
dataset = {
    "features":x,
    "labels":y
}

In [24]:
with open ("anishDataset.pkl","wb") as f:
    pickle.dump(dataset,f)